In [ ]:
import numpy as np
import heston_call_numpy as hc
from mtgl_test import *

In [ ]:
# input: calibrated parameters (x0,r,kappa,mu,eta,V0,rho) for each maturity 
# function: generate option payoff given parameters from the Heston model (question: how to deal with running max when we have different calibrated parameters for different maturities?)

In [ ]:
# question: how do we calculate the running max? Doubly make sure. Recall that we have a new set of calibrated parameters for each maturity.

In [ ]:
# code smoothing kernel
def kernel(rho, x):
    # x can be a matrix or a scalar, rho is a scalar
    return ((rho-1)/2)*(np.float_power(np.abs(x)+1,-rho))

In [ ]:
# test 'kernel'
x = np.array([[1,2],[1,2]])
kernel(3,x) # test passed

In [ ]:
def mtgLoss_vanilla(rho, calibrated_payoff, market_price):
    # group by maturities
    mat_num = len(calibrated_payoff) # number of maturities
    itr, strike_num = calibrated_payoff[0].shape
    vanilla_loss = []
    for i in range(mat_num):
        sum_ls = []
        for j in range(itr):
            # perform element-wise matrix multiplication
            diff = calibrated_payoff[i][j].T.reshape(strike_num,1)-market_price[i][j]
            compute_mat = np.multiply(diff,kernel(rho,diff))  
            sum_ls.append(np.sum(np.multiply(compute_mat,compute_mat)))
        vanilla_loss.append(sum(sum_ls)) # summing loss over all iterations
    vanilla_loss = sum(vanilla_loss) # summing loss over all maturities
    # compute average loss across iterations
    avg_loss = vanilla_loss/(itr*mat_num*strike_num)
    
    return avg_loss

In [ ]:
x = hc.Heston_stock(itr = 10, dt = 1/12, timesteps = 12)[0]
strikes_call = np.arange(0.4,1.4, 0.2)
print(strikes_call)
maturities = range(1,6,2)
print(list(maturities))

In [ ]:
result = hc.price_payoff_coupling(x = x, strikes_call = strikes_call, maturities = maturities, itr = 10, dt = 1/12, timesteps = 12)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 5

mtgLoss_vanilla(rho, calibrated_payoff, market_price)

### Previous Formulation (Garbage Collection)

In [ ]:
# write the function for martingale projection loss
def mtgLoss_pair(rho, calibrated_payoff, market_price, summation = False):
    # group by maturities
    mat_num = len(calibrated_payoff[1]) # number of maturities
    vanilla_loss = []
    for i in range(mat_num):
        # perform element-wise matrix multiplication
        diff = calibrated_payoff[1][i]-market_price[1][i]
        compute_mat = np.multiply(diff,kernel(rho,diff))  
        vanilla_loss.append(np.sum(np.multiply(compute_mat,compute_mat)))
    vanilla_loss = sum(vanilla_loss)
    # exotic options
    diff = calibrated_payoff[0]-market_price[0]
    compute_mat = np.multiply(diff,kernel(rho,diff)) 
    exotic_loss = np.sum(np.multiply(compute_mat,compute_mat))
    
    if summation:
        sum_loss = exotic_loss+vanilla_loss
        print("Total loss of vanilla and exotic options: ", sum_loss)
        return sum_loss
    else:
        print("Loss in vanilla: ", vanilla_loss)
        print("Loss in exotic: ", exotic_loss)
        return (exotic_loss, vanilla_loss)

In [ ]:
# test 'mtgLoss_vanilla'
strikes_call = np.arange(0.4,1.4, 0.2)
print(strikes_call)
maturities = range(1,12,4)
print(list(maturities))

mat_ls_1 = hc.price_option(strikes_call = strikes_call, maturities = maturities, itr = 100, timesteps = 12)
mat_ls_2 = hc.price_option(strikes_call = strikes_call, maturities = maturities, itr = 100, timesteps = 12)

In [ ]:
# they should be a martingale 
rho = 5
mtgLoss_pair(rho, mat_ls_1, mat_ls_2) # but how do we interpret the numerics?

In [ ]:
# train neural network on 59.pt

### Train Neural Network on Call_price59.pt

#### Train on LV Model with four maturities

In [ ]:
# perform martingale test
maturities = range(16, 65, 16)
print(list(maturities))
# load data
stock_traj = np.loadtxt("stock_traj_LV.txt")
print(stock_traj)

In [ ]:
len(stock_traj)

In [ ]:
# select the first monte carlo trial of stock trajectory
stock_traj[0]

In [ ]:
strikes_call = np.arange(0.8, 1.21, 0.02)
result = hc.price_payoff_coupling(x = stock_traj, strikes_call = strikes_call, maturities = maturities, itr = 1000, dt = 1/96, timesteps = 96)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 2

In [ ]:
# Save the list to a file
filename = 'LV_calibrated_payoff.pkl'
with open(filename, 'wb') as file:
    pickle.dump(result[1], file)

In [ ]:
mtgLoss_vanilla(rho, calibrated_payoff, market_price)

In [ ]:
calibrated_payoff[0].shape

In [ ]:
len(calibrated_payoff)

In [ ]:
import pickle

#### Train on LSV Model with two maturities

In [ ]:
rho = 5
lbd = -50
ubd = 50
conf = 95

In [ ]:
# perform martingale test
maturities = range(16, 33, 16)
print(list(maturities))
# load data
stock_traj = np.loadtxt("LSV_calibrated_stock_traj.txt")
print(stock_traj)

In [ ]:
# take the first monte carlo iteration
stock_traj[0]

In [ ]:
strikes_call = np.arange(0.8, 1.21, 0.02)
print(strikes_call)
maturities = range(16, 33, 16)
print(list(maturities))

result = hc.price_payoff_coupling(x = stock_traj, strikes_call = strikes_call, maturities = maturities, itr = 1000, dt = 1/96, timesteps = 96)
calibrated_payoff = result[1]
market_price = result[0] # vanilla
rho = 5

In [ ]:
def mtgLoss_vanilla(rho, calibrated_payoff, market_price):
    # group by maturities
    mat_num = len(calibrated_payoff) # number of maturities
    itr, strike_num = calibrated_payoff[0].shape
    vanilla_loss = []
    for i in range(mat_num):
        sum_ls = []
        for j in range(itr):
            # perform element-wise matrix multiplication
            diff = calibrated_payoff[i][j].T.reshape(strike_num,1)-market_price[i][j]
            compute_mat = np.multiply(diff,kernel(rho,diff))  
            sum_ls.append(np.sum(np.multiply(compute_mat,compute_mat)))
        vanilla_loss.append(sum(sum_ls)) # summing loss over all iterations
    vanilla_loss = sum(vanilla_loss) # summing loss over all maturities
    # compute average loss across iterations
    avg_loss = vanilla_loss/(itr*mat_num*strike_num)
    
    return avg_loss

In [ ]:
calibrated_payoff[0].shape

In [ ]:
calibrated_payoff

In [ ]:
len(calibrated_payoff[0])

In [ ]:
len(calibrated_payoff[0][0])

In [ ]:
vanilla_loss = mtgLoss_vanilla(rho, calibrated_payoff, market_price)
vanilla_loss

In [ ]:
result = np.loadtxt('rho5sig1.txt')

In [ ]:
bond = cutoff(result,conf)
bond

In [ ]:
21*4000*2

In [ ]:
np.sqrt(168000)*vanilla_loss

In [ ]:
mtgl_test(rho,calibrated_payoff, market_price,lbd,ubd,conf,result,vanilla_loss)

In [ ]:
#mtgLoss_vanilla(rho, calibrated_payoff, market_price)

In [ ]:
# q. experiment of the sigma (detect the bump)